# Inferring Topics from IMDB Reviews

In [1]:
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import pandas as pd
import matplotlib.pyplot as plt

## Exploring the Dataset: [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz)

In [2]:
ROOT = './train/pos/'

In [3]:
reviews = []
for file in os.listdir(ROOT):
    path = os.path.join(ROOT, file)
    if os.path.isfile(path):
        with open(path, 'r') as fin:
            reviews.append(fin.read())

In [4]:
len(reviews)

2050

In [5]:
for i in range(3):
    print(reviews[i])
    print('=' * 150)

One of my favourite films first saw it when I was about 10, which probably tells you a lot about the type of humour. Although dated the humour definitely has a charm about it. Expect to see the usual Askey & Murdoch banter so popular in its day, with lots of interesting, quirky co-characters. The lady with the parrot, the couple due to get married and are in trouble from 'her', and my favourite, the stationmaster, "Nobody knows where it comes from ... nobody knows where it goes.." Interestingly the ghost train was written by Arnold Ridley of Dads Army fame (Private Godfrey the medic) Watch it on a rainy Sunday afternoon after your lunch and smile.
Having not seen the films before (and not being able to stand Matt Damon), I was reluctant to go see The Bourne Ultimatum when we were asked to see it for AS Film Studies. <br /><br />However, I was pleasantly surprised that even a film with Damon in it could be enjoyable. <br /><br />Fast fight scenes, crazy motorbike chases and BIG explosio

## Feature Extraction

In [7]:
vect = TfidfVectorizer(stop_words='english')
X = vect.fit_transform(reviews)

pd.DataFrame(X.toarray(), columns=vect.get_feature_names_out())

,00,000,007,0080,00s,01,02,03,05,06,...,álex,álvaro,ángel,äänekoski,åge,écran,émigrés,était,ís,østbye
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## NMF Decomposition

In [8]:
N_TOPICS = 15
nmf = NMF(n_components=N_TOPICS)
W = nmf.fit_transform(X)  # Document-topic matrix
H = nmf.components_       # Topic-term matrix

/usr/local/python/3.12.1/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [9]:
# Top 10 words per topic

words = np.array(vect.get_feature_names_out())
topic_words = pd.DataFrame(np.zeros((N_TOPICS, 10)), index=[f'Topic {i + 1}' for i in range(N_TOPICS)],
                           columns=[f'Word {i + 1}' for i in range(10)]).astype(str)
for i in range(N_TOPICS):
    ix = H[i].argsort()[::-1][:10]
    topic_words.iloc[i] = words[ix]

topic_words

,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10
Topic 1,br,10,scene,ve,plot,high,ll,quite,story,just
Topic 2,movie,movies,saw,watch,fun,recommend,acting,watching,seen,thought
Topic 3,film,films,director,style,characters,seen,character,like,watch,art
Topic 4,season,episode,seasons,episodes,sabrina,characters,shows,watch,eric,ended
Topic 5,life,love,people,young,story,man,time,world,real,way
Topic 6,murder,andrews,man,tierney,noir,john,cast,vance,role,preminger
Topic 7,good,really,just,like,don,think,know,say,people,didn
Topic 8,series,episode,tv,episodes,war,dvd,television,new,trek,sci
Topic 9,tom,jerry,cartoon,invisible,mouse,short,cat,cartoons,butch,lee
Topic 10,disney,cinderella,animation,holes,fairy,mice,king,lion,ball,prince


In [10]:
# Create a topic mapping

topic_mapping = {
    'Topic 4': 'TV',
    'Topic 7': 'War',
    'Topic 8': 'Comedy',
    'Topic 12': 'Book Adaptation',
    'Topic 13': 'Horror',
    'Topic 15': 'Martial Arts / Action'
}

In [11]:
# Recall the document-topic matrix, W

W = pd.DataFrame(W, columns=[f'Topic {i + 1}' for i in range(N_TOPICS)])
W['max_topic'] = W.apply(lambda x: topic_mapping.get(x.idxmax()), axis=1)
W[pd.notnull(W['max_topic'])].head(10)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,max_topic
0,0.001741,0.004687,0.009812,0.009997,0.013075,0.000345,0.007052,0.000495,0.004797,0.000644,0.009302,0.020603,0.022866,0.006139,0.000000,Horror
5,0.000000,0.030372,0.004433,0.000000,0.019343,0.000000,0.000000,0.018503,0.000000,0.000000,0.000000,0.148939,0.004111,0.000000,0.031520,Book Adaptation
6,0.000000,0.014463,0.039191,0.000000,0.010722,0.006606,0.008949,0.000000,0.000000,0.000000,0.052960,0.002264,0.000000,0.000000,0.056054,Martial Arts / Action
12,0.021232,0.003692,0.002968,0.000000,0.014522,0.000057,0.023233,0.007059,0.003790,0.000000,0.003012,0.000000,0.000000,0.000000,0.000000,War
22,0.000000,0.029902,0.017585,0.000000,0.000000,0.000000,0.001509,0.000000,0.000000,0.000000,0.000000,0.000000,0.057154,0.004857,0.000000,Horror
23,0.000000,0.011563,0.030759,0.000000,0.000000,0.000000,0.081428,0.000000,0.000000,0.000000,0.000000,0.081600,0.000000,0.000000,0.005487,Book Adaptation
28,0.000000,0.000000,0.044775,0.001749,0.000211,0.000000,0.019303,0.002226,0.004437,0.012131,0.005789,0.000000,0.020933,0.000000,0.056976,Martial Arts / Action
29,0.019056,0.002916,0.000000,0.345521,0.000000,0.000000,0.006942,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,TV
31,0.000000,0.018698,0.047818,0.000000,0.000000,0.000000,0.003116,0.000000,0.000000,0.000000,0.000000,0.183293,0.000000,0.000000,0.015380,Book Adaptation
38,0.045914,0.020129,0.051653,0.008485,0.000000,0.000000,0.017763,0.000000,0.000000,0.000000,0.000000,0.208025,0.000000,0.014555,0.000000,Book Adaptation


In [12]:
reviews[58]

'James Dickey is a wonderfully descriptive author. When one reads "Deliverance", one is instantly transported into the lush backwoods of the Deep South. When one watches John Boorman\'s film version of the book, one realizes just how accurately he captures the essence of the book. The camera is as descriptive as the narration. The characters are fully realized, and the portrayals are fantastic. I first saw this movie in 1992, after my freshman year of college. I was in a phase where I was watching movies that were all released within a couple of years of my birth in 1973. Among them were "Patton", "Papillon", and "All the President\'s Men"; fine films, all of them. This one was easily the class of the group. That says a lot.'

In [13]:
# Frobenius norm

import numpy as np

print("Frobenius norm and the condition number:")
print(np.linalg.norm([[1,1,1],[3,4,1],[4,1,2]], 'fro'))
print(np.linalg.cond([[1,1,1],[3,4,1],[4,1,2]], 'fro'))


Frobenius norm and the condition number:
7.0710678118654755
13.975424859373685
